In [79]:
import gym
import numpy as np
import gymnasium as gym 


In [80]:
import flappy_bird_gymnasium
import gymnasium as gym

# env = gym.make("FlappyBird-v0", render_mode="human", use_lidar=True)
env = gym.make("FlappyBird-v0",use_lidar=False)
obs, info = env.reset()




In [81]:
obs.shape

(12,)

In [82]:
env.action_space

Discrete(2)

In [83]:
env.reset()
state,_ = env.reset()
done = False
while not done:
    action = int(np.random.choice([0,1]))
    next_state, reward, done, truncated, _ = env.step(action)
    print(action,reward)

0 0.1
1 0.1
1 0.1
1 0.1
0 0.1
1 0.1
0 0.1
1 0.1
1 0.1
1 0.1
0 0.1
0 0.1
1 0.1
1 0.1
0 0.1
1 0.1
1 0.1
0 0.1
0 0.1
0 0.1
1 0.1
1 0.1
0 0.1
1 0.1
1 0.1
0 0.1
0 0.1
0 0.1
1 0.1
0 -0.5
1 -0.5
0 -0.5
0 -0.5
0 -0.5
0 -0.5
1 -0.5
1 -0.5
1 -0.5
0 -0.5
0 -0.5
1 -0.5
1 -0.5
0 -0.5
0 -0.5
0 -0.5
1 -0.5
0 -0.5
1 -0.5
1 -0.5
0 -1


In [84]:
from torch import nn
import torch

In [85]:
class PolicyNet(nn.Module):
    
    def __init__(self,input_dim,output_dim):
        super().__init__()
        self.linear1 = nn.Linear(input_dim,200)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(200,output_dim)
        self.softmax = nn.Softmax(dim=-1)
    
    def forward(self,state):
        ### n
        x = self.linear1(state)
        x = self.relu(x)
        x = self.linear2(x) # n
        x = self.softmax(x) # n
        return x

In [86]:
from torch.distributions import Categorical
import numpy as np
np.bool8 = np.bool_

from torch.optim import AdamW

In [87]:
class Agent:
    
    def __init__(self):
        self.policy_net = PolicyNet(12,2)
        self.optimizer = AdamW(self.policy_net.parameters(),lr=1e-3)
    
    def sample_action(self,state):
        probs = self.policy_net(state) # 4
        if np.random.uniform() < 0.0:
            action = np.random.randint(0,2)
            return action, torch.log(probs[action]+1e-8)
        dist = Categorical(probs)
        action = dist.sample()
        log_prob = dist.log_prob(action)
        return action.item(),log_prob
    
    def update(self,rewards,log_probs):
        ### 一次游戏时间
        ret = []
        adding = 0
        for r in rewards[::-1]:
            adding = adding * 0.99 + r
            ret.insert(0,adding)
        ret = torch.FloatTensor(ret)
        ret = ret - ret.mean()
        ret = ret / (ret.std()+1e-8)
        
        r_log_probs = []
        for r,log_prob in zip(ret,log_probs):
            r_log_probs.append(-r*log_prob)
        r_log_probs = torch.vstack(r_log_probs)
        
        loss = r_log_probs.sum()
        
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        
        return loss

In [88]:
def convert2tensor(state):
    state_arr = torch.FloatTensor(state)
    return state_arr

In [89]:
def train(agent,env):
    success_count = []
    max_size = 1000
    for epoch in range(20000000):
        rewards = []
        log_probs = []
        terminated = False
        success = True
        state,_ = env.reset()
        while not terminated and len(log_probs) < max_size:
            state_arr = convert2tensor(state)
            action, log_prob = agent.sample_action(state_arr)
            next_state, reward, terminated, truncated, _ = env.step(action)
            state = next_state
            rewards.append(reward)
            log_probs.append(log_prob)
        
        loss = agent.update(rewards,log_probs) 
        success_count.append(success)
        
        
        if (epoch+1) % 10 == 0:
            print(f'epoch: {epoch}, loss: {loss}, rewards: {sum(rewards)}, count: {len(rewards)}')

In [90]:
agent = Agent()



In [101]:
env = gym.make("FlappyBird-v0",use_lidar=False)
train(agent,env)

epoch: 9, loss: 0.2699802815914154, rewards: 8.399999999999986, count: 86
epoch: 19, loss: 2.7737550735473633, rewards: 13.499999999999972, count: 128
epoch: 29, loss: 0.7120810151100159, rewards: 23.20000000000007, count: 207
epoch: 39, loss: -1.4867277145385742, rewards: 18.200000000000014, count: 166
epoch: 49, loss: -0.3117551803588867, rewards: 22.700000000000063, count: 202
epoch: 59, loss: -1.4446250200271606, rewards: 8.399999999999986, count: 86
epoch: 69, loss: -1.7559071779251099, rewards: 17.90000000000001, count: 163
epoch: 79, loss: 2.75830078125, rewards: 22.40000000000006, count: 199
epoch: 89, loss: 0.06955179572105408, rewards: 17.90000000000001, count: 163
epoch: 99, loss: -1.6416422128677368, rewards: 27.200000000000113, count: 238
epoch: 109, loss: -2.8319194316864014, rewards: 22.900000000000066, count: 204
epoch: 119, loss: -0.7819856405258179, rewards: 3.8999999999999986, count: 50
epoch: 129, loss: 0.34236666560173035, rewards: 17.90000000000001, count: 163
epo

epoch: 1079, loss: -1.2368907928466797, rewards: 13.499999999999972, count: 128
epoch: 1089, loss: -0.5841783285140991, rewards: 9.399999999999983, count: 96
epoch: 1099, loss: -0.9616409540176392, rewards: 18.300000000000015, count: 167
epoch: 1109, loss: -8.192109107971191, rewards: 25.00000000000008, count: 216
epoch: 1119, loss: -1.6525373458862305, rewards: 18.00000000000001, count: 164
epoch: 1129, loss: 4.068902969360352, rewards: 12.899999999999974, count: 122
epoch: 1139, loss: 2.729628801345825, rewards: 34.70000000000019, count: 295
epoch: 1149, loss: -1.9424107074737549, rewards: 8.699999999999985, count: 89
epoch: 1159, loss: -2.639286518096924, rewards: 8.399999999999986, count: 86
epoch: 1169, loss: -0.5340666174888611, rewards: 13.099999999999973, count: 124
epoch: 1179, loss: -4.05173397064209, rewards: 46.10000000000032, count: 391
epoch: 1189, loss: 0.5795187950134277, rewards: 22.50000000000006, count: 200
epoch: 1199, loss: -1.9965726137161255, rewards: 22.40000000

epoch: 2129, loss: -1.0469598770141602, rewards: 13.199999999999973, count: 125
epoch: 2139, loss: -0.6031270027160645, rewards: 8.399999999999986, count: 86
epoch: 2149, loss: -5.799953460693359, rewards: 8.999999999999984, count: 92
epoch: 2159, loss: -10.1764497756958, rewards: 4.1999999999999975, count: 53
epoch: 2169, loss: -0.3321964144706726, rewards: 9.399999999999983, count: 96
epoch: 2179, loss: -1.1131876707077026, rewards: 18.400000000000016, count: 168
epoch: 2189, loss: 1.068885087966919, rewards: 18.400000000000016, count: 168
epoch: 2199, loss: 0.1368669718503952, rewards: 8.399999999999986, count: 86
epoch: 2209, loss: -1.389510154724121, rewards: 8.399999999999986, count: 86
epoch: 2219, loss: -1.0948586463928223, rewards: 8.699999999999985, count: 89
epoch: 2229, loss: -1.059458613395691, rewards: 8.599999999999985, count: 88
epoch: 2239, loss: 5.390729904174805, rewards: 13.79999999999997, count: 131
epoch: 2249, loss: -1.4755992889404297, rewards: 8.399999999999986

epoch: 3179, loss: -8.46863079071045, rewards: 32.30000000000017, count: 280
epoch: 3189, loss: -0.0033131837844848633, rewards: 12.999999999999973, count: 123
epoch: 3199, loss: 0.9787770509719849, rewards: 13.099999999999973, count: 124
epoch: 3209, loss: -1.3425593376159668, rewards: 74.99999999999993, count: 626
epoch: 3219, loss: -4.670026779174805, rewards: 40.90000000000026, count: 348
epoch: 3229, loss: -4.580491542816162, rewards: 41.30000000000027, count: 352
epoch: 3239, loss: 0.07786056399345398, rewards: 8.399999999999986, count: 86
epoch: 3249, loss: -0.4964662492275238, rewards: 20.500000000000032, count: 180
epoch: 3259, loss: 0.0056766243651509285, rewards: 6.5999999999999925, count: 68
epoch: 3269, loss: 1.207205891609192, rewards: 6.799999999999992, count: 70
epoch: 3279, loss: 0.7054789066314697, rewards: 8.899999999999984, count: 91
epoch: 3289, loss: 2.2515177726745605, rewards: 22.40000000000006, count: 199
epoch: 3299, loss: 0.265078067779541, rewards: 8.3999999

epoch: 4229, loss: -1.1080031394958496, rewards: 9.099999999999984, count: 93
epoch: 4239, loss: 0.44256412982940674, rewards: 8.999999999999984, count: 92
epoch: 4249, loss: 5.0772199630737305, rewards: 8.399999999999986, count: 86
epoch: 4259, loss: -0.4431978166103363, rewards: 3.8999999999999986, count: 50
epoch: 4269, loss: 0.843498706817627, rewards: 12.899999999999974, count: 122
epoch: 4279, loss: -2.1530447006225586, rewards: 12.899999999999974, count: 122
epoch: 4289, loss: 0.6444000005722046, rewards: 6.5999999999999925, count: 68
epoch: 4299, loss: -2.5080888271331787, rewards: 13.599999999999971, count: 129
epoch: 4309, loss: 0.41641509532928467, rewards: 8.399999999999986, count: 86
epoch: 4319, loss: -2.1738357543945312, rewards: 22.600000000000062, count: 201
epoch: 4329, loss: 0.5049083232879639, rewards: 12.899999999999974, count: 122
epoch: 4339, loss: -1.2299829721450806, rewards: 18.500000000000018, count: 169
epoch: 4349, loss: 2.7018136978149414, rewards: 8.39999

...

epoch: 70699, loss: -2.1675543785095215, rewards: 110.89999999999834, count: 913
epoch: 70709, loss: 1.37119460105896, rewards: 12.899999999999974, count: 122
epoch: 70719, loss: 0.010648787021636963, rewards: 122.49999999999785, count: 1000
epoch: 70729, loss: 0.846243143081665, rewards: 13.299999999999972, count: 126
epoch: 70739, loss: -1.8434031009674072, rewards: 15.299999999999972, count: 137
epoch: 70749, loss: 4.349429607391357, rewards: 26.90000000000011, count: 235
epoch: 70759, loss: -1.0880177021026611, rewards: 34.600000000000186, count: 294
epoch: 70769, loss: 1.6704944372177124, rewards: 22.40000000000006, count: 199
epoch: 70779, loss: 1.3106545209884644, rewards: 122.49999999999785, count: 1000
epoch: 70789, loss: -12.874114990234375, rewards: 106.99999999999851, count: 883
epoch: 70799, loss: -13.54542064666748, rewards: 116.69999999999807, count: 962
epoch: 70809, loss: -9.118646621704102, rewards: 83.89999999999954, count: 697
epoch: 70819, loss: 4.257366180419922, 

epoch: 71729, loss: -1.3359346389770508, rewards: 40.90000000000026, count: 348
epoch: 71739, loss: -1.8095488548278809, rewards: 40.90000000000026, count: 348
epoch: 71749, loss: -3.2486531734466553, rewards: 24.90000000000008, count: 215
epoch: 71759, loss: 4.378616809844971, rewards: 8.399999999999986, count: 86
epoch: 71769, loss: 0.3974481225013733, rewards: 4.899999999999995, count: 60
epoch: 71779, loss: 0.17880240082740784, rewards: 53.10000000000039, count: 443
epoch: 71789, loss: -1.959162712097168, rewards: 96.89999999999897, count: 800
epoch: 71799, loss: -1.351187825202942, rewards: 96.89999999999897, count: 800
epoch: 71809, loss: 4.413675785064697, rewards: 26.90000000000011, count: 235
epoch: 71819, loss: 0.3033159077167511, rewards: 4.4999999999999964, count: 56
epoch: 71829, loss: -2.044851779937744, rewards: 50.800000000000374, count: 429
epoch: 71839, loss: -4.577502250671387, rewards: 122.49999999999785, count: 1000
epoch: 71849, loss: -15.99254322052002, rewards: 

epoch: 72759, loss: -4.795592308044434, rewards: 80.89999999999971, count: 667
epoch: 72769, loss: -0.5181204080581665, rewards: 122.49999999999785, count: 1000
epoch: 72779, loss: 4.024538040161133, rewards: 8.399999999999986, count: 86
epoch: 72789, loss: -4.208961486816406, rewards: 26.90000000000011, count: 235
epoch: 72799, loss: 0.21716946363449097, rewards: 4.299999999999997, count: 54
epoch: 72809, loss: 2.062282085418701, rewards: 3.8999999999999986, count: 50
epoch: 72819, loss: 3.2956106662750244, rewards: 9.499999999999982, count: 97
epoch: 72829, loss: -0.4264216423034668, rewards: 82.8999999999996, count: 687
epoch: 72839, loss: -1.9006757736206055, rewards: 4.399999999999997, count: 55
epoch: 72849, loss: -0.2779277265071869, rewards: 4.699999999999996, count: 58
epoch: 72859, loss: -18.36713409423828, rewards: 111.19999999999833, count: 916
epoch: 72869, loss: -4.0966949462890625, rewards: 122.49999999999785, count: 1000
epoch: 72879, loss: -7.258112907409668, rewards: 

epoch: 73789, loss: 2.6088879108428955, rewards: 6.199999999999994, count: 64
epoch: 73799, loss: 2.5895004272460938, rewards: 122.49999999999785, count: 1000
epoch: 73809, loss: 2.3942010402679443, rewards: 122.49999999999785, count: 1000
epoch: 73819, loss: 1.021870732307434, rewards: 40.90000000000026, count: 348
epoch: 73829, loss: 3.154627561569214, rewards: 8.399999999999986, count: 86
epoch: 73839, loss: 0.09170675277709961, rewards: 122.49999999999785, count: 1000
epoch: 73849, loss: 4.784360408782959, rewards: 122.49999999999785, count: 1000
epoch: 73859, loss: 0.13446760177612305, rewards: 122.49999999999785, count: 1000
epoch: 73869, loss: -0.2986879348754883, rewards: 68.90000000000022, count: 574
epoch: 73879, loss: -0.13248825073242188, rewards: 122.49999999999785, count: 1000
epoch: 73889, loss: -2.3064515590667725, rewards: 122.49999999999785, count: 1000
epoch: 73899, loss: 1.0582129955291748, rewards: 20.600000000000033, count: 181
epoch: 73909, loss: 2.93669795989990

KeyboardInterrupt: 

In [92]:
# def sample_action(self,state):
#     probs = self.policy_net(state) # 4
#     if np.random.uniform() < 0.3:
#         action = np.random.randint(0,2)
#         return action, torch.log(probs[action]+1e-8)
#     dist = Categorical(probs)
#     action = dist.sample()
#     log_prob = dist.log_prob(action)
#     return action.item(),log_prob

# # 替换方法
# import types
# agent.sample_action = types.MethodType(sample_action, agent)

In [ ]:
import time
def visualize_agent(env, agent, num_episodes=5):
    """
    渲染显示智能体的行动
    """
    env = gym.make('FlappyBird-v0', render_mode='human',use_lidar=False)  # 创建可视化环境
    
    for episode in range(num_episodes):
        state_tuple = env.reset()
        state = state_tuple[0] if isinstance(state_tuple, tuple) else state_tuple
        total_reward = 0
        steps = 0
        done = False
        
        print(f"\nEpisode {episode + 1}")
        
        while not done:
            env.render()  # 渲染当前状态
            
            # 将状态转换为one-hot编码
    
            
            # 使用训练好的策略选择动作
            with torch.no_grad():
                if np.random.random() < 0.0:
                    action = np.random.randint(0, 4)
                else:
                    state_tensor = torch.FloatTensor(state)
                    probs = agent.policy_net(state_tensor)
                    action = probs.argmax().item()  # 使用最可能的动作
            
            # 执行动作
            step_result = env.step(action)
            if len(step_result) == 4:
                next_state, reward, done, _ = step_result
            else:
                next_state, reward, terminated, truncated, _ = step_result
                done = terminated or truncated
            
            total_reward += reward
            steps += 1
            state = next_state
            
            # 添加小延迟使动作更容易观察
            time.sleep(0.01)
        
        print(f"Episode finished after {steps} steps. Total reward: {total_reward}")
    
    env.close()

# 在主程序最后添加：
if __name__ == "__main__":    
    # 训练完成后显示智能体行动
    print("\nVisualizing trained agent behavior...")
    env = gym.make('FlappyBird-v0',render_mode='human',use_lidar=False)
    visualize_agent(env, agent)


Visualizing trained agent behavior...

Episode 1
Episode finished after 7128 steps. Total reward: 880.9000000001037

Episode 2


In [100]:
env.close()